# Git - Control de Versiones
*Sofía García Salas*

*2021-03-02*

***

Estas son mis notas sobre la clase de MIT [The Missing Semester](https://missing.csail.mit.edu/2020/version-control/).

## Control de versiones
`Git` es un sistema de control de versiones que permite documentar los cambios a código u otro documentos y folders. También facilitan el trabajo en equipo.

Estos sistemas almacenan la historia de cambios tomando una fotografía al estado de los archivos y folders. No solo tienen la historia de los cambios a estos archivos sino que también almacenan metadata (e.g. quién realizó un cambio, en qué momento).

### Utilidad
Sirve no solo en proyectos grandes, sino en proyecto personales para poder revisar versiones antiguos del código, ver por qué se cambio algo (viendo commit messages), trabajar en paralelo con distintas branches para el desarrollo, etc. 

### Git
Se ha vuelto el sistema de control de versiones más popular.

Se interfaz no es muy amigable con el usuario, por lo que el profesor recomienda empezar aprendiendo su diseño e ideas. Al entender esto, será más fácil utilizar la interfaz y resolver problemas.

## Modelo de datos de Git
### Modelo de folders y archivos
Es una colección de archivos y folders dentro de un directorio maestro.
Un folder es denominado como `tree` y un archivo como `blob`.

```
root
|-foo (tree)
| |-bar.txt (blob)
|-baz.txt

```

El folder `root`es el directorio al que se le está dando seguimiento.

### Modelo de historia
Una forma de crear un modelo de historia es tomar una fotografía de los folders y archivos en el directorio. La historia se vuelve una secuencia lineal de estas fotografías. Es muy similar a tener copias de folders con marcas de tiempo.

Git utilizado un modelo más complejo llamado **Directed acyclic graph**. 

Con este modelo uno puede separar versiones (`fork`) para trabajar en ramas (`branches`) en paralelo sin interferencia; por ejemplo, trabajar es una nueva característica y en paralelo arreglar un desperfecto en el código en producción. 
Estas ramas después se pueden unir (`merge`) y la nueva fotografía tendrá como referencia las dos versiones que se unieron.



![history](img/history.jpg)

Es pseudocódigo, el modelo de archivos e historia de `Git`se puede explicar de la siguiente manera:

**Tipos de estructuras:**
```
type blob = array<byte>    # file = coleccion de bytes

type tree = map<string, tree | blob> # folder = mapea un nombre de directorio a contenido (archivos o folders)

type commit = struct{           # metadata con la información de la fotografía
    parents: array<commit>
    author: string
    message: string
    snapshot: tree  # esta es una referencia al objeto no el objeto en si
```
**Almacenamiento en disco:**los objetos son los que Git almacena en disco
```
type object = blog | tree | commit

objects = map<string, objects>

# para almacenar en disco se utiliza el hash del objeto
def store(o)
    id = sha1(o) # hash del objeto
    objects[id] = o

}

df load(id)
    return objects[id]
```
**Referencias**:
mapea referencias de un nombre legible para el humano con su hash respectivo
```
references = map<string, string>
```

La historia y los hash son inmutables, pero las referencias sí se pueden cambiar.
En forma general, todos los comandos de `Git` son manipulaciones de los datos de los *objetos* o de las *referencia*

## Demostración

Esta sera una demostración de cómo utilizar comandos de git, pero no de como hacer la inicializacion para no alterar el repositorio ya existente.

Al visualizar el directorio `.git` podemos `refs` y `objects` que son donde `Git` almacena la información de los repositorios.

In [6]:
%sx ls ../../.git

['branches',
 'config',
 'description',
 'HEAD',
 'hooks',
 'index',
 'info',
 'logs',
 'objects',
 'packed-refs',
 'refs']

El comando `git help` se puede utilizar para obtener información sobre `git`. También se puede utilizar `git help <comando>` para tener información adicional del comando específico.

In [8]:
%sx git help

['usage: git [--version] [--help] [-C <path>] [-c <name>=<value>]',
 '           [--exec-path[=<path>]] [--html-path] [--man-path] [--info-path]',
 '           [-p | --paginate | -P | --no-pager] [--no-replace-objects] [--bare]',
 '           [--git-dir=<path>] [--work-tree=<path>] [--namespace=<name>]',
 '           <command> [<args>]',
 '',
 'These are common Git commands used in various situations:',
 '',
 'start a working area (see also: git help tutorial)',
 '   clone             Clone a repository into a new directory',
 '   init              Create an empty Git repository or reinitialize an existing one',
 '',
 'work on the current change (see also: git help everyday)',
 '   add               Add file contents to the index',
 '   mv                Move or rename a file, a directory, or a symlink',
 '   restore           Restore working tree files',
 '   rm                Remove files from the working tree and from the index',
 '   sparse-checkout   Initialize and modify the spar

El comando `git status` se puede ver información de la historia y el estatus actual del repositorio.

In [2]:
%sx git status

['On branch main',
 "Your branch is up to date with 'origin/main'.",
 '',
 'nothing to commit, working tree clean']

El staging area es dónde le decimos a git qué cambios incluir en la próxima fotografía que tome.
Podemos ver cómo cambia el status cuándo se crea un nuevo archivo:

In [3]:
%sx echo 'archivo de ejemplo para git' > archivo_ejemplo.txt
%sx git status

['On branch main',
 "Your branch is up to date with 'origin/main'.",
 '',
 'Untracked files:',
 '  (use "git add <file>..." to include in what will be committed)',
 '\tarchivo_ejemplo.txt',
 '',
 'nothing added to commit but untracked files present (use "git add" to track)']

Y cómo el status cambia al agregarlo al tracking de git

In [4]:
%sx git add archivo_ejemplo.txt
%sx git status

['On branch main',
 "Your branch is up to date with 'origin/main'.",
 '',
 'Changes to be committed:',
 '  (use "git restore --staged <file>..." to unstage)',
 '\tnew file:   archivo_ejemplo.txt',
 '']

Para tomar la fotografía, se utiliza el comando `git commit`

In [5]:
%sx git commit

['[main 83644be] Agregar archivo de ejemplo',
 ' 1 file changed, 1 insertion(+)',
 ' create mode 100644 tareas/02-tarea/archivo_ejemplo.txt']

Se puede utilizar el comando `git log` para ver la historia de todos los cambios. Aquí utilizaré -1 para solo mostrar el último commit

In [13]:
%sx git log -1

['commit 83644be39f4251ddf141c8c894d3644c37a0af27',
 'Author: Sofi GS <19873735+sofi-gt@users.noreply.github.com>',
 'Date:   Sat Feb 6 12:09:09 2021 -0600',
 '',
 '    Agregar archivo de ejemplo']

Para ver el contenido de un commit se puede utilizar el comando `git cat-file` con el hash

In [14]:
%sx git cat-file -p 83644be

['tree 56e4e6215bca85188f3d6b9d504abf0f01507667',
 'parent 946f83da08279b6d47ad9ef58562bf5b838dc313',
 'author Sofi GS <19873735+sofi-gt@users.noreply.github.com> 1612634949 -0600',
 'committer Sofi GS <19873735+sofi-gt@users.noreply.github.com> 1612634949 -0600',
 '',
 'Agregar archivo de ejemplo']

`git add` y `git commit` están separados para poder tener mayor flexibilidad y organización. Yo puedo decidir qué archivos se incluyen en cada commit